In [18]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import nest_asyncio
nest_asyncio.apply()

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"            

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (12, 8),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import pandas as pd
import numpy as np

import datetime
import pytz
NY_tz = pytz.timezone("America/New_York") 
CHI_tz = pytz.timezone("America/Chicago") 
UTC_tz = pytz.timezone("UTC") 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import datetime
import ujson as json
from pathlib import Path
from typing import Iterable, Optional

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import QuantLib as ql
import tqdm


def _df_to_parquet(df: pd.DataFrame, path: Path, *, compression: Optional[str] = "zstd"):
    out = df.copy()
    for col in out.columns:
        s = out[col]

    if s.dtype == "object" and s.map(type).nunique() > 1:
        out[col] = s.astype(str)

    path.parent.mkdir(parents=True, exist_ok=True)
    tbl = pa.Table.from_pandas(out, preserve_index=False)
    pq.write_table(tbl, path, compression=compression)

In [20]:
PATH_STORE = {
    "2Y": Path("./TRACE_2Y"),
    "3Y": Path("./TRACE_3Y"),
    "5Y": Path("./TRACE_5Y"),
    "7Y": Path("./TRACE_7Y"),
    "10Y": Path("./TRACE_10Y"),
    "20Y": Path("./TRACE_20Y"),
    "30Y": Path("./TRACE_30Y"),
}

In [21]:
from FinraFetcher import FinraDataFetcher
ff = FinraDataFetcher(
    debug_verbose=True,
	info_verbose=True,
	error_verbose=True,
)

In [ ]:
start = datetime.datetime(2025, 9, 1)
end = datetime.datetime(2025, 9, 17)
bdays = pd.date_range(start=start, end=end, freq="1b")
bdays

DatetimeIndex(['2025-09-17'], dtype='datetime64[ns]', freq='B')

In [23]:
for bd in bdays:
    benchmarks = ["2Y", "3Y", "5Y", "7Y", "10Y", "20Y", "30Y"]
    intraday_dict = ff.fetch_historcal_trace_trade_history_by_cusip_v2(
        start_date=bd,
        end_date=bd,
        benchmark_terms=benchmarks,
        # session_timeout_minutes=8,
        # max_connections=8,
        # max_in_flight=3,
        force_close=False,
    )
    for bm in benchmarks:
        PATH_STORE[bm].mkdir(parents=True, exist_ok=True)
        output_path = PATH_STORE[bm] / f"{bd.strftime('%Y-%m-%d')}.parquet"
        _df_to_parquet(intraday_dict[bm], output_path)

2025-09-27 14:19:05,488 - FinraDataFetcher - INFO - TRACE - FINRA totals probe took: 149.156s
INFO	Task(Task-1) FinraDataFetcher:FinraFetcher.py:_run()- TRACE - FINRA totals probe took: 149.156s
2025-09-27 14:19:05,489 - FinraDataFetcher - DEBUG - TRACE - totals: {'2Y': 51795, '3Y': 51232, '5Y': 112806, '7Y': 43529, '10Y': 96630, '20Y': 16277, '30Y': 37963}
DEBUG	Task(Task-1) FinraDataFetcher:FinraFetcher.py:_run()- TRACE - totals: {'2Y': 51795, '3Y': 51232, '5Y': 112806, '7Y': 43529, '10Y': 96630, '20Y': 16277, '30Y': 37963}
2025-09-27 14:41:47,479 - FinraDataFetcher - INFO - TRACE - Fetch All Took: 1361.990s
INFO	Task(Task-1) FinraDataFetcher:FinraFetcher.py:_run()- TRACE - Fetch All Took: 1361.990s
2025-09-27 14:41:47,481 - FinraDataFetcher - INFO - TRACE - Total Time Elapsed: 1511.303s
INFO	Task(Task-2) FinraDataFetcher:FinraFetcher.py:fetch_historcal_trace_trade_history_by_cusip_v2()- TRACE - Total Time Elapsed: 1511.303s


In [ ]:
intraday_dict = ff.fetch_historcal_trace_trade_history_by_cusip_v3(
	start_date=datetime.datetime(2025, 9, 26),
	end_date=datetime.datetime(2025, 9, 26),
	benchmark_terms=["10Y"],
)